In [1]:
import os
import math
import time
import gzip
import urllib
import collections
import numpy as np
import tensorflow as tf

In [2]:
DEFAULT_SOURCE_URL = 'http://yann.lecun.com/exdb/mnist/'

In [3]:
def _read32(bytestream):
    dt = np.dtype(np.uint32).newbyteorder('>')
    return np.frombuffer(bytestream.read(4), dtype=dt)[0]

In [4]:
def extract_images(f):
    """Extract the images into a 4D uint8 numpy array [index, y, x, depth]."""
    print('Extracting', f.name)
    with gzip.GzipFile(fileobj=f) as bytestream:
        magic = _read32(bytestream)
        if magic != 2051:
            raise ValueError('Invalid magic number %d in MNIST image file: %s' % (magic, f.name))
        num_images = _read32(bytestream)
        rows = _read32(bytestream)
        cols = _read32(bytestream)
        buf = bytestream.read(rows * cols * num_images)
        data = np.frombuffer(buf, dtype=np.uint8)
        data = data.reshape(num_images, rows, cols, 1)
        return data

In [5]:
def dense_to_one_hot(labels_dense, num_classes):
    """Convert class labels from scalars to one-hot vectors."""
    num_labels = labels_dense.shape[0]
    index_offset = np.arange(num_labels) * num_classes
    labels_one_hot = np.zeros((num_labels, num_classes))
    labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
    return labels_one_hot

In [6]:
def extract_labels(f, one_hot=False, num_classes=10):
    """Extract the labels into a 1D uint8 numpy array [index]."""
    print('Extracting', f.name)
    with gzip.GzipFile(fileobj=f) as bytestream:
        magic = _read32(bytestream)
        if magic != 2049:
            raise ValueError('Invalid magic number %d in MNIST label file: %s' % (magic, f.name))
        num_items = _read32(bytestream)
        buf = bytestream.read(num_items)
        labels = np.frombuffer(buf, dtype=np.uint8)
        if one_hot:
            return dense_to_one_hot(labels, num_classes)
        return labels

In [7]:
class DataSet(object):
    """Container class for a dataset."""

    def __init__(self, images, labels, fake_data=False, one_hot=False, dtype=tf.float32, reshape=True, seed=None):
        """Construct a DataSet."""
        seed1, seed2 = tf.get_seed(seed)
        # If op level seed is not set, use whatever graph level seed is returned
        np.random.seed(seed1 if seed is None else seed2)
        dtype = tf.as_dtype(dtype).base_dtype
        if dtype not in (tf.uint8, tf.float32):
            raise TypeError('Invalid image dtype %r, expected uint8 or float32' % dtype)
        if fake_data:
            self._num_examples = 10000
            self.one_hot = one_hot
        else:
            assert images.shape[0] == labels.shape[0], ('images.shape: %s labels.shape: %s' % (images.shape, labels.shape))
            self._num_examples = images.shape[0]

            # Convert shape from [num examples, rows, columns, depth]
            # to [num examples, rows*columns] (assuming depth == 1)
            if reshape:
                assert images.shape[3] == 1
                images = images.reshape(images.shape[0], images.shape[1] * images.shape[2])
            if dtype == tf.float32:
                # Convert from [0, 255] -> [0.0, 1.0].
                images = images.astype(np.float32)
                images = np.multiply(images, 1.0 / 255.0)
        self._images = images
        self._labels = labels
        self._epochs_completed = 0
        self._index_in_epoch = 0

    @property
    def images(self):
        return self._images

    @property
    def labels(self):
        return self._labels

    @property
    def num_examples(self):
        return self._num_examples

    @property
    def epochs_completed(self):
        return self._epochs_completed

    def next_batch(self, batch_size, fake_data=False, shuffle=True):
        """Return the next `batch_size` examples from this data set."""
        if fake_data:
            fake_image = [1] * 784
            if self.one_hot:
                fake_label = [1] + [0] * 9
            else:
                fake_label = 0
            return [fake_image for _ in range(batch_size)], [fake_label for _ in range(batch_size)]
        start = self._index_in_epoch
        # Shuffle for the first epoch
        if self._epochs_completed == 0 and start == 0 and shuffle:
            perm0 = np.arange(self._num_examples)
            np.random.shuffle(perm0)
            self._images = self.images[perm0]
            self._labels = self.labels[perm0]
        # Go to the next epoch
        if start + batch_size > self._num_examples:
            # Finished epoch
            self._epochs_completed += 1
            # Get the rest examples in this epoch
            rest_num_examples = self._num_examples - start
            images_rest_part = self._images[start:self._num_examples]
            labels_rest_part = self._labels[start:self._num_examples]
            # Shuffle the data
            if shuffle:
                perm = np.arange(self._num_examples)
                np.random.shuffle(perm)
                self._images = self.images[perm]
                self._labels = self.labels[perm]
            # Start next epoch
            start = 0
            self._index_in_epoch = batch_size - rest_num_examples
            end = self._index_in_epoch
            images_new_part = self._images[start:end]
            labels_new_part = self._labels[start:end]
            return np.concatenate((images_rest_part, images_new_part), axis=0), np.concatenate((labels_rest_part, labels_new_part), axis=0)
        else:
            self._index_in_epoch += batch_size
            end = self._index_in_epoch
            return self._images[start:end], self._labels[start:end]

In [8]:
def urlretrieve_with_retry(url, filename=None):
    return urllib.request.urlretrieve(url, filename)

In [9]:
def maybe_download(filename, work_directory, source_url):
    """Download the data from source url, unless it's already here."""
    if not tf.gfile.Exists(work_directory):
        tf.gfile.MakeDirs(work_directory)
    filepath = os.path.join(work_directory, filename)
    if not tf.gfile.Exists(filepath):
        temp_file_name, _ = urlretrieve_with_retry(source_url)
        tf.gfile.Copy(temp_file_name, filepath)
        with tf.gfile.GFile(filepath) as f:
            size = f.size()
        print('Successfully downloaded', filename, size, 'bytes.')
    return filepath

In [10]:
Datasets = collections.namedtuple('Datasets', ['train', 'validation', 'test'])

In [11]:
def read_data_sets(train_dir, fake_data=False, one_hot=False, dtype=tf.float32, reshape=True, validation_size=5000, seed=None, source_url=DEFAULT_SOURCE_URL):
    if fake_data:
        def fake():
            return DataSet([], [], fake_data=True, one_hot=one_hot, dtype=dtype, seed=seed)

        train = fake()
        validation = fake()
        test = fake()
        return Datasets(train=train, validation=validation, test=test)

    if not source_url:  # empty string check
        source_url = DEFAULT_SOURCE_URL

    TRAIN_IMAGES = 'train-images-idx3-ubyte.gz'
    TRAIN_LABELS = 'train-labels-idx1-ubyte.gz'
    TEST_IMAGES = 't10k-images-idx3-ubyte.gz'
    TEST_LABELS = 't10k-labels-idx1-ubyte.gz'

    local_file = maybe_download(TRAIN_IMAGES, train_dir, source_url + TRAIN_IMAGES)
    with tf.gfile.Open(local_file, 'rb') as f:
        train_images = extract_images(f)

    local_file = maybe_download(TRAIN_LABELS, train_dir, source_url + TRAIN_LABELS)
    with tf.gfile.Open(local_file, 'rb') as f:
        train_labels = extract_labels(f, one_hot=one_hot)

    local_file = maybe_download(TEST_IMAGES, train_dir, source_url + TEST_IMAGES)
    with tf.gfile.Open(local_file, 'rb') as f:
        test_images = extract_images(f)

    local_file = maybe_download(TEST_LABELS, train_dir, source_url + TEST_LABELS)
    with tf.gfile.Open(local_file, 'rb') as f:
        test_labels = extract_labels(f, one_hot=one_hot)

    if not 0 <= validation_size <= len(train_images):
        raise ValueError('Validation size should be between 0 and {}. Received: {}.'.format(len(train_images), validation_size))

    validation_images = train_images[:validation_size]
    validation_labels = train_labels[:validation_size]
    train_images = train_images[validation_size:]
    train_labels = train_labels[validation_size:]

    options = dict(dtype=dtype, reshape=reshape, seed=seed)

    train = DataSet(train_images, train_labels, **options)
    validation = DataSet(validation_images, validation_labels, **options)
    test = DataSet(test_images, test_labels, **options)

    return Datasets(train=train, validation=validation, test=test)

In [12]:
learning_rate = 0.01
max_steps = 50000
hidden1_units = 128
hidden2_units = 32
batch_size = 100
input_data_dir = 'input_data'
log_dir = 'logs/fully_connected_feed'
fake_data = False

In [13]:
if tf.gfile.Exists(log_dir):
    tf.gfile.DeleteRecursively(log_dir)
tf.gfile.MakeDirs(log_dir)

In [14]:
data_sets = read_data_sets(input_data_dir, fake_data)

Extracting input_data/train-images-idx3-ubyte.gz
Extracting input_data/train-labels-idx1-ubyte.gz
Extracting input_data/t10k-images-idx3-ubyte.gz
Extracting input_data/t10k-labels-idx1-ubyte.gz


In [15]:
# The MNIST dataset has 10 classes, representing the digits 0 through 9.
NUM_CLASSES = 10

# The MNIST images are always 28x28 pixels.
IMAGE_SIZE = 28
IMAGE_PIXELS = IMAGE_SIZE * IMAGE_SIZE

In [16]:
images_placeholder = tf.placeholder(tf.float32, shape=(batch_size, IMAGE_PIXELS), name='images')
labels_placeholder = tf.placeholder(tf.int32, shape=(batch_size), name='labels')

In [17]:
# Hidden 1
weights = tf.Variable(
    tf.truncated_normal([IMAGE_PIXELS, hidden1_units], stddev=1.0 / math.sqrt(float(IMAGE_PIXELS))),
    name='weights')
biases = tf.Variable(tf.zeros([hidden1_units]), name='biases')
hidden1 = tf.nn.relu(tf.matmul(images_placeholder, weights) + biases)
# Hidden 2
weights = tf.Variable(
    tf.truncated_normal([hidden1_units, hidden2_units], stddev=1.0 / math.sqrt(float(hidden1_units))),
    name='weights')
biases = tf.Variable(tf.zeros([hidden2_units]), name='biases')
hidden2 = tf.nn.relu(tf.matmul(hidden1, weights) + biases)
# Linear
weights = tf.Variable(
    tf.truncated_normal([hidden2_units, NUM_CLASSES], stddev=1.0 / math.sqrt(float(hidden2_units))),
    name='weights')
biases = tf.Variable(tf.zeros([NUM_CLASSES]), name='biases')
logits = tf.matmul(hidden2, weights) + biases

In [18]:
labels = tf.to_int64(labels_placeholder)
loss = tf.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)

In [19]:
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
global_step = tf.Variable(0, name='global_step', trainable=False)
train_op = optimizer.minimize(loss, global_step=global_step)

In [20]:
prediction = tf.argmax(logits, 1)
correct = tf.equal(prediction, labels)
eval_correct = tf.reduce_sum(tf.cast(correct, tf.int32))

In [21]:
init = tf.global_variables_initializer()

In [22]:
saver = tf.train.Saver()

In [23]:
def fill_feed_dict(data_set, images_pl, labels_pl):
    """Fills the feed_dict for training the given step."""
    images_feed, labels_feed = data_set.next_batch(batch_size, fake_data)
    feed_dict = {images_pl: images_feed, labels_pl: labels_feed}
    return feed_dict

In [24]:
def do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_set):
    """Runs one evaluation against the full epoch of input_data."""
    # And run one epoch of eval.
    true_count = 0  # Counts the number of correct predictions.
    steps_per_epoch = data_set.num_examples // batch_size
    num_examples = steps_per_epoch * batch_size
    for step in range(steps_per_epoch):
        feed_dict = fill_feed_dict(data_set, images_placeholder, labels_placeholder)
        true_count += sess.run(eval_correct, feed_dict=feed_dict)
    precision = float(true_count) / num_examples
    print('Num examples: %d  Num correct: %d  Precision @ 1: %0.04f' % (num_examples, true_count, precision))

In [25]:
sess = tf.Session()
sess.run(init)

for step in range(max_steps):
    start_time = time.time()

    feed_dict = fill_feed_dict(data_sets.train, images_placeholder, labels_placeholder)

    _, loss_value = sess.run([train_op, loss], feed_dict=feed_dict)

    duration = time.time() - start_time

    if step % 100 == 0:
        print('Step %d: loss = %.2f (%.3f sec)' % (step, loss_value, duration))

    if (step + 1) % 1000 == 0 or (step + 1) == max_steps:
        checkpoint_file = os.path.join(log_dir, 'model.ckpt')
        saver.save(sess, checkpoint_file, global_step=step)
        # Evaluate against the training set.
        print('Training Data Eval:')
        do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_sets.train)
        # Evaluate against the validation set.
        print('Validation Data Eval:')
        do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_sets.validation)
        # Evaluate against the test set.
        print('Test Data Eval:')
        do_eval(sess, eval_correct, images_placeholder, labels_placeholder, data_sets.test)

Step 0: loss = 2.32 (0.141 sec)
Step 100: loss = 2.14 (0.002 sec)
Step 200: loss = 1.90 (0.002 sec)
Step 300: loss = 1.58 (0.002 sec)
Step 400: loss = 1.20 (0.002 sec)
Step 500: loss = 0.93 (0.002 sec)
Step 600: loss = 0.81 (0.004 sec)
Step 700: loss = 0.77 (0.002 sec)
Step 800: loss = 0.61 (0.002 sec)
Step 900: loss = 0.61 (0.002 sec)
Training Data Eval:
Num examples: 55000  Num correct: 47047  Precision @ 1: 0.8554
Validation Data Eval:
Num examples: 5000  Num correct: 4318  Precision @ 1: 0.8636
Test Data Eval:
Num examples: 10000  Num correct: 8656  Precision @ 1: 0.8656
Step 1000: loss = 0.53 (0.009 sec)
Step 1100: loss = 0.44 (0.100 sec)
Step 1200: loss = 0.61 (0.002 sec)
Step 1300: loss = 0.35 (0.005 sec)
Step 1400: loss = 0.43 (0.003 sec)
Step 1500: loss = 0.35 (0.006 sec)
Step 1600: loss = 0.44 (0.002 sec)
Step 1700: loss = 0.46 (0.002 sec)
Step 1800: loss = 0.40 (0.002 sec)
Step 1900: loss = 0.22 (0.002 sec)
Training Data Eval:
Num examples: 55000  Num correct: 49412  Precisi

Step 14100: loss = 0.08 (0.002 sec)
Step 14200: loss = 0.09 (0.002 sec)
Step 14300: loss = 0.07 (0.101 sec)
Step 14400: loss = 0.16 (0.002 sec)
Step 14500: loss = 0.14 (0.002 sec)
Step 14600: loss = 0.10 (0.002 sec)
Step 14700: loss = 0.26 (0.002 sec)
Step 14800: loss = 0.21 (0.002 sec)
Step 14900: loss = 0.18 (0.002 sec)
Training Data Eval:
Num examples: 55000  Num correct: 52886  Precision @ 1: 0.9616
Validation Data Eval:
Num examples: 5000  Num correct: 4809  Precision @ 1: 0.9618
Test Data Eval:
Num examples: 10000  Num correct: 9561  Precision @ 1: 0.9561
Step 15000: loss = 0.10 (0.010 sec)
Step 15100: loss = 0.16 (0.002 sec)
Step 15200: loss = 0.21 (0.002 sec)
Step 15300: loss = 0.10 (0.002 sec)
Step 15400: loss = 0.18 (0.103 sec)
Step 15500: loss = 0.25 (0.002 sec)
Step 15600: loss = 0.18 (0.007 sec)
Step 15700: loss = 0.08 (0.002 sec)
Step 15800: loss = 0.07 (0.002 sec)
Step 15900: loss = 0.08 (0.003 sec)
Training Data Eval:
Num examples: 55000  Num correct: 52923  Precision @

Num examples: 55000  Num correct: 53818  Precision @ 1: 0.9785
Validation Data Eval:
Num examples: 5000  Num correct: 4860  Precision @ 1: 0.9720
Test Data Eval:
Num examples: 10000  Num correct: 9686  Precision @ 1: 0.9686
Step 28000: loss = 0.04 (0.010 sec)
Step 28100: loss = 0.11 (0.002 sec)
Step 28200: loss = 0.08 (0.002 sec)
Step 28300: loss = 0.07 (0.002 sec)
Step 28400: loss = 0.04 (0.002 sec)
Step 28500: loss = 0.05 (0.002 sec)
Step 28600: loss = 0.09 (0.103 sec)
Step 28700: loss = 0.10 (0.002 sec)
Step 28800: loss = 0.15 (0.002 sec)
Step 28900: loss = 0.11 (0.002 sec)
Training Data Eval:
Num examples: 55000  Num correct: 53858  Precision @ 1: 0.9792
Validation Data Eval:
Num examples: 5000  Num correct: 4861  Precision @ 1: 0.9722
Test Data Eval:
Num examples: 10000  Num correct: 9700  Precision @ 1: 0.9700
Step 29000: loss = 0.09 (0.009 sec)
Step 29100: loss = 0.19 (0.002 sec)
Step 29200: loss = 0.05 (0.002 sec)
Step 29300: loss = 0.12 (0.002 sec)
Step 29400: loss = 0.11 (0.0

Step 41500: loss = 0.02 (0.002 sec)
Step 41600: loss = 0.05 (0.002 sec)
Step 41700: loss = 0.06 (0.002 sec)
Step 41800: loss = 0.02 (0.106 sec)
Step 41900: loss = 0.04 (0.002 sec)
Training Data Eval:
Num examples: 55000  Num correct: 54325  Precision @ 1: 0.9877
Validation Data Eval:
Num examples: 5000  Num correct: 4876  Precision @ 1: 0.9752
Test Data Eval:
Num examples: 10000  Num correct: 9762  Precision @ 1: 0.9762
Step 42000: loss = 0.01 (0.011 sec)
Step 42100: loss = 0.02 (0.004 sec)
Step 42200: loss = 0.03 (0.003 sec)
Step 42300: loss = 0.05 (0.003 sec)
Step 42400: loss = 0.05 (0.003 sec)
Step 42500: loss = 0.03 (0.002 sec)
Step 42600: loss = 0.11 (0.002 sec)
Step 42700: loss = 0.02 (0.002 sec)
Step 42800: loss = 0.04 (0.003 sec)
Step 42900: loss = 0.03 (0.127 sec)
Training Data Eval:
Num examples: 55000  Num correct: 54365  Precision @ 1: 0.9885
Validation Data Eval:
Num examples: 5000  Num correct: 4880  Precision @ 1: 0.9760
Test Data Eval:
Num examples: 10000  Num correct: 

In [28]:
from PIL import Image

steps_per_epoch = data_sets.test.num_examples // batch_size
for step in range(steps_per_epoch):
    images_feed, labels_feed = data_sets.test.next_batch(batch_size)
    feed_dict = {images_placeholder: images_feed, labels_placeholder: labels_feed}
    prediction_value = sess.run(prediction, feed_dict=feed_dict)
    incorrect_index = np.where(prediction_value != labels_feed)
    # print(incorrect_index)
    for i in incorrect_index[0]:
        pred = prediction_value[i]
        label = labels_feed[i]
        # print(pred, label)
        image = images_feed[i]
        image = np.multiply(image, 255.0)
        image = image.astype(np.uint8)
        image = image.reshape(28, 28)
        im = Image.fromarray(image)
        if not os.path.exists(os.path.join(input_data_dir, 'incorrect', str(label))):
            os.makedirs(os.path.join(input_data_dir, 'incorrect', str(label)))
        filename = os.path.join(input_data_dir, 'incorrect', str(label),
                                '%d_%d_%d.bmp' % (pred, np.mean(image) * 1e6, np.std(image) * 1e6))
        im.save(filename)